# Imports


In [19]:
# -*- coding: utf-8 -*-

# import necessary libraries
import pandas as pd
import os
import glob
  
from os import path
from requests import get
import numpy as np
import re
import unicodedata
import prepare
import spacy
import dexplot as dxp

# Acquire

In [14]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))

df = pd.DataFrame()  
# loop over the list of csv files
for f in csv_files:
    
    # read the csv file
    temp = pd.read_csv(f)
    
    # print the location and filename
    temp['region']=f.split("\\")[-1][-13:-11]

    df= pd.concat([df, temp])

df['rank'] = df.index + 1
df['top_25'] = np.where(df['rank'] < 26, 1, 0)
df.description = df.description.fillna('no description')


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9536 entries, 0 to 159
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           9536 non-null   object
 1   title              9536 non-null   object
 2   publishedAt        9536 non-null   object
 3   channelTitle       9536 non-null   object
 4   categoryId         9536 non-null   int64 
 5   trending_date      9536 non-null   object
 6   tags               9536 non-null   object
 7   view_count         9536 non-null   int64 
 8   likes              9536 non-null   int64 
 9   comment_count      9536 non-null   int64 
 10  thumbnail_link     9536 non-null   object
 11  comments_disabled  9536 non-null   bool  
 12  ratings_disabled   9536 non-null   bool  
 13  description        9536 non-null   object
 14  duration           9536 non-null   object
 15  captions           9536 non-null   bool  
 16  region             9536 non-null   object
 

In [16]:
df = df[df.duplicated(['video_id'], keep=False) == True].sort_values(by='rank').drop_duplicates(['video_id']).reset_index(drop=True)

In [17]:
df

,video_id,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,duration,captions,region,rank,top_25
0,sWwHWK7OYJ8,[ENG][아형✪하이라이트] (여자)아이들 완전체 떴다↗ 멋짐 폭발♥ 5인 5색 토...,2022-10-29T13:26:44Z,아는형님 Knowingbros,24,22.01.11,아는 형님|형님|형님 학교|미연|민니|소연|우기|슈화|아이들|여자아이들|강호동|이수...,1098919,19090,861,https://i.ytimg.com/vi/sWwHWK7OYJ8/default.jpg,False,False,[아형✪하이라이트] (여자)아이들 완전체 떴다↗ 멋짐 폭발♥ 5인 5색 토크 모음....,PT21M21S,True,KR,1,1
1,_CCTtb_wSuA,5 histoires flippantes volume 3 : Éteignez la ...,2022-10-31T18:47:36Z,Mcfly et Carlito,24,22.01.11,mcfly|carlito|golden moustache|fatshow|badass|...,2217807,182434,7282,https://i.ytimg.com/vi/_CCTtb_wSuA/default.jpg,False,False,Un grand merci à nos 3 fabuleux guests : Gaëll...,PT1H12M7S,False,CA,1,1
2,AcznL5daQvA,Fear PIRATE CHAMPION's Spear! Clash of Clans S...,2022-11-01T08:00:04Z,Clash of Clans,20,22.01.11,clash of clans|COC|Clash of Clans Gameplay|Cla...,2258144,44366,938,https://i.ytimg.com/vi/AcznL5daQvA/default.jpg,False,False,Ahoy! ⚓ A new month of Season Challenges is he...,PT41S,False,DE,1,1
3,ZYO1Z4Ajdvw,ЗА МНОЙ СЛЕДИЛИ?! ТОТ САМЫЙ ЛЕС В ЯПОНИИ! Ghos...,2022-10-31T20:54:41Z,Дима Масленников (Pognali Show),24,22.01.11,ЗА МНОЙ СЛЕДИЛИ?! ТОТ САМЫЙ ЛЕС В ЯПОНИИ|ТОТ С...,3262953,312903,14437,https://i.ytimg.com/vi/ZYO1Z4Ajdvw/default.jpg,False,False,Оформи подписку на VK Музыку за 0 рублей до ко...,PT31M2S,False,RU,1,1
4,bbl7oKjOw1U,シーズン15 アプデ徹底検証！RE-45が絶対炎上レベルの強さｗｗ ランページ大人気間違いな...,2022-11-01T20:11:42Z,TIE Ru,20,22.02.11,TIE Ru|ゲーム実況|PCゲーム|Apex Legends|エーペックスレジェンズ|Ap...,313545,4899,596,https://i.ytimg.com/vi/bbl7oKjOw1U/default.jpg,False,False,►►『Twitter』 - https://twitter.com/Poulmt►►お仕事/...,PT16M5S,False,JP,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,kLuzD9SoiEU,Encontramos las ESCALERAS de HACKERS MÁS ALTAS...,2022-10-21T17:03:29Z,RaptorGamer,20,22.01.11,raptor|raptorgamer|gamer|RAPTOR|invictor|escal...,1902762,78690,1712,https://i.ytimg.com/vi/kLuzD9SoiEU/default.jpg,False,False,Oiii!!! Que tal todos!!?? esta vez estaremos e...,PT18M19S,False,MX,199,0
2015,WRr0EDTrDx8,【プロ】バレー第1回目のゲストが本気すぎたんだけど、、、,2022-10-23T11:00:11Z,スカイピース,22,22.01.11,スカイピース,1976426,48816,3060,https://i.ytimg.com/vi/WRr0EDTrDx8/default.jpg,False,False,​ ​@コムドット @中町兄妹 @ESPOIR TRIBE-エスポワール•トライブ- ...,PT31M9S,False,JP,200,0
2016,j-3Tpmi7CCk,"Pabllo Vittar, Gloria Groove - AMEIANOITE (Off...",2022-10-21T02:59:03Z,Pabllo Vittar,10,22.01.11,[none],1529830,114260,11625,https://i.ytimg.com/vi/j-3Tpmi7CCk/default.jpg,False,False,Watch more Pabllo Vittar YouTube videos here /...,PT2M46S,False,BR,200,0
2017,iDBRKLjfJS4,망한 모텔 사서 한 달에 4천만원 버는 38살 사장님,2022-10-09T09:34:15Z,탐구생활 - 돈이 되는 삶의 이야기,22,22.01.11,[none],1835677,12011,1015,https://i.ytimg.com/vi/iDBRKLjfJS4/default.jpg,False,False,광주와 담양에서 호텔을 운영하는 여인욱 대표님입니다탐구생활: tamgu-life@n...,PT19M51S,False,KR,200,0


In [18]:

def acquire ():
    # use glob to get all the csv files 
    # in the folder
    path = os.getcwd()
    csv_files = glob.glob(os.path.join(path, "*.csv"))
    
    df = pd.DataFrame()  
    # loop over the list of csv files
    for f in csv_files:
        
        # read the csv file
        temp = pd.read_csv(f)
        
        # print the location and filename
        temp['region']=f.split("\\")[-1][-13:-11]

        df= pd.concat([df, temp])

    df['rank'] = df.index + 1
    df['top_25'] = np.where(df['rank'] < 26, 1, 0)
    df.description = df.description.fillna('no description')
    df = df[df.duplicated(['video_id'], keep=False) == True].sort_values(by='rank').drop_duplicates(['video_id']).reset_index(drop=True)
    df.region = np.where(df.region == 'IN', 'IND', df.region)
    return df

In [60]:
test=acquire()

In [67]:
df = acquire()

# Functions


In [80]:
def clean_duration(duration):
    """ 
    Purpose:
        
    ---
    Parameters:
        
    ---
    Returns:
    
    """
    if ('S' not in duration) & ('M' not in duration):
        duration += '00M00S'
    elif 'M' not in duration:
        duration = list(duration)
        print('stop')
        print(list(duration))
        duration.insert(-3, '00M') if len(duration) > 6 else duration.insert(-2, '00M')
        duration = ''.join(duration)
    elif 'S' not in duration:
        duration += '00S'

    if 'H' in duration:
        duration = int(duration.split('H')[0].split('T')[1]) * 3600 + int(duration.split('H')[1].split('M')[0]) * 60 + int(duration.split('M')[1][:-1])
    elif duration.__contains__('M'):
        duration = int(duration.split('M')[0].split('T')[1]) * 60 + int(duration.split('M')[1][:-1])
    else:
        duration = int(duration[-3:-1])

    return duration

def clean_text(text):
    """ 
    Purpose:
        to clean text input into function by removing duplicate words, punctuations, and other things
    ---
    Parameters:
        text: a string
    ---
    Returns:
        tokens: a set of words found in the input text
    """

    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    tokens = set()
    
    for token in doc:
        if token.pos_ not in ['SYM', 'PUNCT', 'DET']:
            tokens.add(token.text)

    for ent in doc.ents:
        tokens.add(ent.text)

    return tokens
    
def title_in_desc(df):
    df['title_in_description'] = 0
    for row in df.index:
        title = df.iloc[row]['title']
        description = df.iloc[row]['description']
                    
        if title in description:
                df.loc[row, 'title_in_description'] = 1
    return df
    
def title_in_tags (df):
    df['title_in_tags'] = 0
    for row in df.index:
        title = df.iloc[row]['title']
        tags = df.iloc[row]['tags']
            
    if title in tags:
            df.loc[row, 'title_in_tags'] = 1
    return df
def pct_tags(df):
    
    df['pct_tags_in_description'] = 0
    for row in df.index:
        counter = 0
        tags = df.iloc[row]['cleaned_tags']
        description = df.iloc[row]['cleaned_desc']

        for tag in tags:
            if tag in description:
                counter += 1
        pct = counter/len(tags)
        df.loc[row, 'pct_tags_in_description'] = pct

    return df
    
def create_bank(text):
    """ 
    Purpose:
        to clean text input into function by removing duplicate words, punctuations, and other things
    ---
    Parameters:
        text: a string
    ---
    Returns:
        tokens: a set of words found in the input text
    """

    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    tokens = list()
    
    for token in doc:
        if token.pos_ not in ['SYM', 'PUNCT', 'DET']:
            tokens.append(token.text)

    for ent in doc.ents:
        tokens.append(ent.text)

    return tokens
    
def create_vaults(df):        #these vaults are to create the word clouds for each portion
    word_vault = list()
    for row in df.index:
        word_vault.extend(df.iloc[row].word_bank)

    top_25_words = list()
    for row in df[df.top_25==1].index:
        top_25_words.extend(df[df.top_25==1].iloc[row].word_bank)

    outside_25_words = list()
    for row in df[df.top_25!=1].index:
        outside_25_words.extend(df[df.top_25!=1].iloc[row].word_bank)

    return word_vault, top_25_words, outside_25_words

In [9]:
def prepare_youtube():
    
    df = acquire()
    
    # creates ranking for each video based off of country index
    df.categoryId = df.categoryId.astype('object')
    df.publishedAt = pd.to_datetime(df.publishedAt, utc=True)
    df.trending_date = '22.2.11'
    df.trending_date = pd.to_datetime(df.trending_date, format='%y.%d.%m', utc=True)
    df['age']=(df.trending_date - df.publishedAt)
    df['engagement'] = (df.likes + df.comment_count * 4 )/df.view_count
    df['sponsored'] = np.where(df.description.str.contains('sponsor'), 1, 0)
    df['duration'] = df['duration'].apply(lambda x:clean_duration(x))
    #countes number of tags given to video BEFORE stripping out extraneous things
    df['num_of_tags'] = df.tags.str.split('|').str.len()
    #gets rid of separator
    df.tags = df.tags.str.replace('|'," ")
    #import the create bank function first
    df['word_bank']= df['description'].apply(lambda x: create_bank(x))
    #import clean_text function first
    df['cleaned_tags'] =  df['tags'].apply(lambda x: clean_text(x))
    df['cleaned_desc'] = df['description'].apply(lambda x: clean_text(x))
    df = title_in_desc(df)
    df = title_in_tags(df)
    df = pct_tags(df)
    #making categorid into actual category titles
    #all categoryId optain from youtube website
    df.categoryId = df.categoryId.map({1: 'Film_Animation', 2: 'Autos_Vehicles',10:'Music',15: 'Pets_Animals'
                                       ,17:'Sports',18:'Short_Movies',19:'Travel_Events',20:'Gaming',21:'Videoblogging',
                                       22:'People_Blogs',23:'Comedy',24:'Entertainment',25:'News_Politics',
                                       26:'Howto_Style',27: 'Education',28: 'Science_Technology', 
                                       29:'Nonprofits_Activism',30:'Movies',31:'Anime/Animation',32:'Action/Adventure',
                                       33:'Classics',34:'Comedy',35:'Documentary',36:'Drama',37:'Family',38:'Foreign',
                                       39:'Horror',40: 'Sci-Fi/Fantasy', 41: 'Thriller', 42:'Shorts',43:'Shows',44:'Trailers'})
    return df

In [81]:
df = prepare_youtube()

stop
['P', 'T', '3', '8', 'S']
stop
['P', 'T', '5', '0', 'S']
stop
['P', 'T', '4', '1', 'S']
stop
['P', 'T', '4', '3', 'S']
stop
['P', 'T', '5', '5', 'S']
stop
['P', 'T', '3', '8', 'S']
stop
['P', 'T', '2', '6', 'S']
stop
['P', 'T', '4', '5', 'S']
stop
['P', 'T', '4', '1', 'S']
stop
['P', 'T', '2', '8', 'S']
stop
['P', 'T', '4', '3', 'S']
stop
['P', 'T', '5', '7', 'S']
stop
['P', 'T', '5', '4', 'S']
stop
['P', 'T', '5', '6', 'S']
stop
['P', 'T', '2', '1', 'S']
stop
['P', 'T', '4', '0', 'S']
stop
['P', 'T', '4', '1', 'S']
stop
['P', 'T', '3', '0', 'S']
stop
['P', 'T', '2', '9', 'S']
stop
['P', 'T', '1', 'H', '4', 'S']
stop
['P', 'T', '5', '1', 'S']
stop
['P', 'T', '4', '1', 'S']
stop
['P', 'T', '4', '0', 'S']
stop
['P', 'T', '3', '5', 'S']
stop
['P', 'T', '3', '5', 'S']
stop
['P', 'T', '2', '7', 'S']
stop
['P', 'T', '3', '0', 'S']
stop
['P', 'T', '3', '5', 'S']
stop
['P', 'T', '2', '5', 'S']
stop
['P', 'T', '3', '5', 'S']
stop
['P', 'T', '3', '3', 'S']
stop
['P', 'T', '4', '1', 'S']
sto

/var/folders/x7/9w9mh0fd73zg4jr8l_9v9h840000gn/T/ipykernel_18590/3837152935.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.tags = df.tags.str.replace('|'," ")


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9536 entries, 0 to 159
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   video_id                 9536 non-null   object             
 1   title                    9536 non-null   object             
 2   publishedAt              9536 non-null   datetime64[ns, UTC]
 3   channelTitle             9536 non-null   object             
 4   categoryId               9536 non-null   object             
 5   trending_date            9536 non-null   datetime64[ns, UTC]
 6   tags                     9536 non-null   object             
 7   view_count               9536 non-null   int64              
 8   likes                    9536 non-null   int64              
 9   comment_count            9536 non-null   int64              
 10  thumbnail_link           9536 non-null   object             
 11  comments_disabled        9536 n

In [84]:
#converts the age category timedelta into hours .. so the videos are x hours old now 
df.age = (df.age.dt.days * 24) + (df.age.dt.seconds/3600)


In [96]:
df['title_lengths'] = df["title"].apply(lambda x: len(x))
df['desc_lengths'] = df["description"].apply(lambda x: 0 if pd.isnull(x) else len(x))
df['tags_length'] = df['tags'].apply(lambda x: len(x.replace('|', '')) if x != '[none]' else 0)


In [97]:
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)
df.to_pickle('prepared.pkl')

In [2]:
testy = prepare.final_prep()

/Users/sinao/codeup-data-science/top_200/prepare.py:254: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.tags = df.tags.str.replace('|'," ")


In [3]:
pesty = pd.read_pickle('prepared.pkl')

In [17]:
pesty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9536 entries, 0 to 9535
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   video_id                 9536 non-null   object             
 1   title                    9536 non-null   object             
 2   publishedAt              9536 non-null   datetime64[ns, UTC]
 3   channelTitle             9536 non-null   object             
 4   categoryId               9536 non-null   object             
 5   trending_date            9536 non-null   datetime64[ns, UTC]
 6   tags                     9536 non-null   object             
 7   view_count               9536 non-null   int64              
 8   likes                    9536 non-null   int64              
 9   comment_count            9536 non-null   int64              
 10  thumbnail_link           9536 non-null   object             
 11  comments_disabled        9536 

In [21]:
pesty.info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9536 entries, 0 to 9535
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   video_id                 9536 non-null   object             
 1   title                    9536 non-null   object             
 2   publishedAt              9536 non-null   datetime64[ns, UTC]
 3   channelTitle             9536 non-null   object             
 4   categoryId               9536 non-null   object             
 5   trending_date            9536 non-null   datetime64[ns, UTC]
 6   tags                     9536 non-null   object             
 7   view_count               9536 non-null   int64              
 8   likes                    9536 non-null   int64              
 9   comment_count            9536 non-null   int64              
 10  thumbnail_link           9536 non-null   object             
 11  comments_disabled        9536 

AttributeError: 'NoneType' object has no attribute 'to_datafram'

In [22]:
pesty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9536 entries, 0 to 9535
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   video_id                 9536 non-null   object             
 1   title                    9536 non-null   object             
 2   publishedAt              9536 non-null   datetime64[ns, UTC]
 3   channelTitle             9536 non-null   object             
 4   categoryId               9536 non-null   object             
 5   trending_date            9536 non-null   datetime64[ns, UTC]
 6   tags                     9536 non-null   object             
 7   view_count               9536 non-null   int64              
 8   likes                    9536 non-null   int64              
 9   comment_count            9536 non-null   int64              
 10  thumbnail_link           9536 non-null   object             
 11  comments_disabled        9536 

 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   video_id                 9536 non-null   object             
 1   title                    9536 non-null   object             
 2   publishedAt              9536 non-null   datetime64
 3   channelTitle             9536 non-null   object             
 4   categoryId               9536 non-null   object             
 5   trending_date            9536 non-null   datetime64
 6   tags                     9536 non-null   object             
 7   view_count               9536 non-null   int64              
 8   likes                    9536 non-null   int64              
 9   comment_count            9536 non-null   int64              
 10  thumbnail_link           9536 non-null   object             
 11  comments_disabled        9536 non-null   bool               
 12  ratings_disabled         9536 non-null   bool               
 13  description              9536 non-null   object             
 14  duration                 9536 non-null   int64              
 15  captions                 9536 non-null   bool               
 16  region                   9536 non-null   object             
 17  rank                     9536 non-null   int64              
 18  top_25                   9536 non-null   int64              
 19  age                      9536 non-null   float64            
 20  engagement               9536 non-null   float64            
 21  sponsored                9536 non-null   int64              
 22  num_of_tags              9536 non-null   int64              
 23  word_bank                9536 non-null   object             
 24  cleaned_tags             9536 non-null   object             
 25  cleaned_desc             9536 non-null   object             
 26  title_in_description     9536 non-null   int64              
 27  title_in_tags            9536 non-null   int64              
 28  pct_tags_in_description  9536 non-null   float64            
 29  title_lengths            9536 non-null   int64              
 30  desc_lengths             9536 non-null   int64              
 31  tags_length              9536 non-null   int64  

In [29]:
pd.read_clipboard().to_markdown()

'|    | #   | Column                  | Non-Null       | Count    | Dtype      |\n|---:|:----|:------------------------|:---------------|:---------|:-----------|\n|  0 | --- | ------                  | -------------- | -----    |            |\n|  1 | 0   | video_id                | 9536           | non-null | object     |\n|  2 | 1   | title                   | 9536           | non-null | object     |\n|  3 | 2   | publishedAt             | 9536           | non-null | datetime64 |\n|  4 | 3   | channelTitle            | 9536           | non-null | object     |\n|  5 | 4   | categoryId              | 9536           | non-null | object     |\n|  6 | 5   | trending_date           | 9536           | non-null | datetime64 |\n|  7 | 6   | tags                    | 9536           | non-null | object     |\n|  8 | 7   | view_count              | 9536           | non-null | int64      |\n|  9 | 8   | likes                   | 9536           | non-null | int64      |\n| 10 | 9   | comment_count 